# Impedance Calibration 

In [47]:
from pathlib import Path
import time
import pandas as pd
import geopandas as gpd
import numpy as np
import modeling_turns
from stochopy.optimize import minimize
from stochastic_optimization import objective_function

In [48]:
fp = Path.home() / 'Documents/BikewaySimData/Projects/gdot'

In [49]:
#import network links
nodes = gpd.read_file(fp/"networks/reconciled_network.gpkg",layer='nodes')
links = gpd.read_file(fp/"networks/reconciled_network.gpkg",layer='links_w_signals_elevation')

In [50]:
#change oneway back to true/false (it saves as a str)
links['oneway'] = links['oneway'] == '1'

#recalculate link lengths
links['length_ft'] = links.length


## Impedance Function 2
- Link Specific:
    - Average Grade (%grade)
    - Vehicle Seperation from OSM/ARC Inventory (1 = None, 2 = Bike Lane, 3 = MUP/Curb protected bike lanes)
    - Number of lanes from HERE ()
- Turn Specific
    - Unsignalized left/straight across roads with higher than tertiary classification (0 or 1)
    - Signalized intersection left/straight (0 or 1)

## Applying Link Costs
---
Dict keys must correspond to column names in links GeoDataFrame. Multiple dicts can be passed to pseudo_df the impacts of changing impedances. The links cost function is of this format:
$$ C_e = \frac{l_e*60^2}{s*5280} * (1-\sum \beta_i x_{i,e}) $$

where:
- $e$ is an edge/link in network graph $G$ with V vertices/nodes and E edges/links
- $l_e$ is the length of the link in feet
- $\beta$ is the impedance coefficient for attribute $i$
- $X_{i,e}$ is the value of attribute $i$ for link $e$
- $s$ is the assumed average speed of the cyclist in mph

Notes:
- Negative attributes **decrease** impedance  
- Positive attributes **increase** impedance
- **Negative link costs are not allowed**
- Time to traverse a link has already been calculated in the prepare_network function

### Create pseudo graph for modeling turns

In [52]:
pseudo_df[pseudo_df['source_linkid']==54818]

,source_A,source_B,target_A,target_B,source_linkid,source_azimuth,source_reverse_link,target_linkid,target_azimuth,target_reverse_link,azimuth_change,turn_type,source,target
301267,1581270385,1581270420,1581270420,69430434,54818.0,33.4,False,46430.0,259.0,True,225.6,left,"(1581270385, 1581270420)","(1581270420, 69430434)"
301270,1581270385,1581270420,1581270420,2909454241,54818.0,33.4,False,47571.0,79.0,True,45.6,right,"(1581270385, 1581270420)","(1581270420, 2909454241)"


In [59]:
pseudo_df['remove'].value_counts()

False    238631
True        249
Name: remove, dtype: int64

In [35]:
attrs = ['linkid','link_type','highway',
         'vehicle_separation','speedlimit_range_mph',
         'lanes_per_direction','up_grade','down_grade','length_ft']
df_edges = df_edges.merge(links[attrs],on='linkid')

# Deal with signals
Perform two merges and use the source/target reverse link columns to determine which signal ID to keep.
- For the source link, use signal_B if reverse == False else signal_A
- For the target link, use signal_A if reverse == False else signal_B

In [38]:
source = pseudo_df[['source_linkid','source_reverse_link']].merge(links,left_on='source_linkid',right_on='linkid',how='left')
pseudo_df['source_signal'] = np.where(source['source_reverse_link'], source['signal_A'], source['signal_B'])

target = pseudo_df[['target_linkid','target_reverse_link']].merge(links,left_on='target_linkid',right_on='linkid',how='left')
pseudo_df['target_signal'] = np.where(target['target_reverse_link']==False, target['signal_A'], target['signal_B'])

## Adding cross streets
- Only look at roads for now
- Filter to left/right turns per source linkid per direction
- Take the highest road classification and assign it as the cross street road classification

In [39]:
highway_order = {
    'trunk': 0,
    'trunk_link': 1,
    'primary': 2,
    'primary_link': 3,
    'secondary': 4,
    'secondary_link': 5,
    'tertiary': 6,
    'tertiary_link': 7,
    'unclassified': 8,
    'residential': 9
}
highway_order = pd.Series(highway_order)
highway_order = highway_order.reset_index()
highway_order.columns = ['highway','order']

In [40]:
# pseudo_df = pd.merge(pseudo_df,source_attr,left_on='source_linkid',right_index=True,how='left')
# pseudo_df = pd.merge(pseudo_df,target_attr,left_on='target_linkid',right_index=True,how='left')

In [42]:
#remove straight and backwards
cond1 = pseudo_df['turn_type'].isin(['left','right'])
#only road to road for now
cond2 = (pseudo_df['source_link_type'] == 'road') & (pseudo_df['target_link_type'] == 'road')
cross_streets = pseudo_df[cond1 & cond2]

#use groupby to find the max target_highway order
cross_streets = cross_streets.groupby(['source_linkid','source_A','source_B'])['target_highway_order'].min()
cross_streets.name = 'cross_street'

#add to main df
pseudo_df = pd.merge(pseudo_df,cross_streets,left_on=['source_linkid','source_A','source_B'],right_index=True,how='left')

#change numbers back to normal
pseudo_df['cross_street_order'] = pseudo_df['cross_street']
pseudo_df['cross_street'] = pseudo_df['cross_street'].map(highway_order.set_index('order')['highway'])

### Classify signalized/unsiganlized turns?

In [43]:
signalized = pseudo_df['source_signal'] == pseudo_df['target_signal']
left_or_straight =  pseudo_df['turn_type'].isin(['left','straight'])
both_road = (pseudo_df['source_link_type'] == 'road') & (pseudo_df['target_link_type'] == 'road')
cross_street = pseudo_df['cross_street_order'] <= 8

pseudo_df.loc[signalized & left_or_straight & both_road,'signalized_left_straight'] = True
pseudo_df.loc[pseudo_df['signalized_left_straight'].isna(),'signalized_left_straight'] = False

pseudo_df.loc[-signalized & left_or_straight & both_road & cross_street,'unsignalized_left_straight_nonlocal'] = True
pseudo_df.loc[pseudo_df['unsignalized_left_straight_nonlocal'].isna(),'unsignalized_left_straight_nonlocal'] = False

In [44]:
from shapely.ops import MultiLineString

#add geo
link_geo = dict(zip(links['linkid'],links['geometry']))
pseudo_df['src_geo'] = pseudo_df['source_linkid'].map(link_geo)
pseudo_df['trgt_geo'] = pseudo_df['target_linkid'].map(link_geo)
pseudo_df['geometry'] = pseudo_df[['src_geo','trgt_geo']].apply(lambda row: MultiLineString([row['src_geo'],row['trgt_geo']]),axis=1)

pseudo_df.drop(columns=['src_geo','trgt_geo'],inplace=True)
pseudo_df = gpd.GeoDataFrame(pseudo_df,crs=links.crs)

pseudo_df['source'] = pseudo_df['source'].astype(str)
pseudo_df['target'] = pseudo_df['target'].astype(str)

#check results (may need a smaller road network to test on)
pseudo_df.to_file(Path.home()/'Downloads/testing.gpkg',layer='cross_streets')

In [45]:
#have position of beta next to name of variable
beta_links = {
    0 : 'up_grade',
    1 : 'vehicle_separation',
    2 : 'speedlimit'
}

beta_turns = {
    3 : 'signalized_left_straight',
    4 : 'unsignalized_left_straight_nonlocal'
}

#customize this function to change impedance formula
#TODO streamline process of trying out new impedance functions
def link_impedance_function(betas,beta_links,links):
    #prevent mutating the original links gdf
    links = links.copy()
    
    links['attr_multiplier'] = 0

    for key, item in beta_links.items():
        links['attr_mulitplier'] = links['attr_mulitplier'] + (betas[key] * links[item])

    links['link_cost'] = links['length_ft'] * (1 + links['attr_multiplier'])
    return links

def turn_impedance_function(betas,beta_turns,pseudo_df):
    #use beta coefficient to calculate turn cost
    base_turn_cost = 30 # from Lowry et al 2016 DOI: http://dx.doi.org/10.1016/j.tra.2016.02.003
    # turn_costs = {
    #     'left': betas[1] * base_turn_cost,
    #     'right': betas[1] * base_turn_cost,
    #     'straight': betas[1] * base_turn_cost
    # }
    #pseudo_df['turn_cost'] = pseudo_df['turn_type'].map(turn_costs)

    pseudo_df = pseudo_df.copy()

    pseudo_df['attr_multiplier'] = 0

    for key, item in beta_turns.items():
        pseudo_df['attr_multiplier'] = links


    return pseudo_df

In [ ]:
#%% prepare link dataframe
links['bike'] = links['bl'] + links['pbl'] + links['mu']
links['bike'] = links['bike'] >= 1

cost_columns = ['linkid','bike','length_ft']#,'up-grade','down-grade','length_ft']
df_edges = df_edges.merge(links[cost_columns],on='linkid')

# df_edges['grade'] = np.nan
# df_edges.loc[df_edges['reverse_link'],'grade'] = df_edges['down-grade']
# df_edges.loc[~df_edges['reverse_link'],'grade'] = df_edges['up-grade']
# #ignore downs
# df_edges.loc[df_edges['grade']<0,'grade'] = 0
# df_edges.drop(columns=['up-grade','down-grade','bearing'],inplace=True)

In [ ]:
#import matched traces
export_fp = Path.home() / 'Documents/BikewaySimData/Projects/gdot/gps_traces'

import pickle
with (export_fp/'test_matches.pkl').open('rb') as fh:
    matched_traces = pickle.load(fh)
matched_traces

#matched_traces = gpd.read_file(export_fp/'test_matches.gpkg')

In [ ]:
matched_traces['linkids'][3]

In [ ]:
#fix set
import ast
matched_traces['linkids'] = matched_traces['linkids'].apply(lambda x: eval(x))

In [ ]:
#drop loops
matched_traces = matched_traces.loc[matched_traces['start']!=matched_traces['end']]

In [ ]:
args = (df_edges,pseudo_df,pseudo_G,matched_traces,False)

In [ ]:
import stochastic_optimization
from importlib import reload
reload(stochastic_optimization)

In [ ]:
# source = 68294161
# target = 2400730083

# pseudo_G, virtual_edges = modeling_turns.add_virtual_links(pseudo_df,pseudo_G,source,[target])   

# virtual_edges

# pseudo_G.out_edges(target)
# #pseudo_G.in_edges((5416154182, 2400730083))

# list(pseudo_G.in_edges(target))[0]

# test = nx.ego_graph(pseudo_G,source,4)
# test.edges()

# import networkx as nx
# test_target = (5318092552,5416166514)

# length, node_list = nx.single_source_dijkstra(pseudo_G,source,test_target,weight='weight')
# node_list

# pseudo_G = modeling_turns.remove_virtual_edges(pseudo_G,virtual_edges)

# import stochastic_optimization
# from importlib import reload
# reload(stochastic_optimization)
# reload(modeling_turns)

# betas = [1.14593853, 0.60739776]
# val, merged = stochastic_optimization.objective_function(betas,*args)

# merged[1].set_geometry('geometry_modeled').set_crs('epsg:2240').explore()

In [ ]:
pseudo_df.columns

In [ ]:
source = 68175462
target = 69214484

import networkx as nx
nx.single_source_dijkstra(pseudo_G,source,target,weight='weight')

In [ ]:
import stochastic_optimization
from importlib import reload
reload(stochastic_optimization)

start = time.time()
bounds = [[0,5],[0,5]]
x = minimize(stochastic_optimization.objective_function, bounds, args=args, method='pso')
end = time.time()
print(f'Took {(end-start)/60/60} hours')
#results[segment_filepath] = (x.x,x.fun)

Need to re-create routes using the coefficients so we can do vizualization

In [ ]:
import stochastic_optimization
from importlib import reload
reload(stochastic_optimization)

betas = np.array([0.09231109, 2.35131751])
args = (df_edges,pseudo_df,pseudo_G,matched_traces,False,True)
test = stochastic_optimization.objective_function(betas,*args)

# Visualization

In [ ]:
test.columns


test['percent_detour'] = (((test['length_ft']-test['shortest_length_ft'])/test['shortest_length_ft'])*100).round(1)


In [ ]:
import pandas as pd
trip_and_user = pd.read_pickle(export_fp/'trip_and_user.pkl')

test_merge = test.merge(trip_and_user,on='tripid')

In [ ]:
tripid = test.loc[test['overlap']<0.2,'tripid'].sample(1).item()
tripid

In [ ]:
row['starttime']

In [ ]:
import folium
import geopandas as gpd
from folium.plugins import MarkerCluster, PolyLineTextPath
from folium.map import FeatureGroup

def visualize(test_merge,tripid):


     gdf = test_merge.copy()

     gdf.set_geometry("geometry",inplace=True)
     gdf.set_crs("epsg:2240",inplace=True)

     # Your GeoDataFrames
     chosen_path = gdf.loc[gdf['tripid']==tripid,['tripid','geometry']]
     shortest_path = gdf.loc[gdf['tripid']==tripid,['tripid','shortest_geo']].set_geometry('shortest_geo').set_crs(gdf.crs)
     intersection = gdf.loc[gdf['tripid']==tripid,['tripid','shortest_intersect_geo']].set_geometry('shortest_intersect_geo').set_crs(gdf.crs)
     modeled_path = gdf.loc[gdf['tripid']==tripid,['tripid','geometry_modeled']].set_geometry('geometry_modeled').set_crs(gdf.crs)

     #start point
     start_N = gdf.loc[gdf['tripid']==tripid,'start'].item()
     start_pt = nodes.to_crs('epsg:4326').loc[nodes['N']==start_N,'geometry'].item()

     #end point
     end_N = gdf.loc[gdf['tripid']==tripid,'end'].item()
     end_pt = nodes.to_crs('epsg:4326').loc[nodes['N']==end_N,'geometry'].item()

     # reproj
     x_mean = chosen_path.to_crs(epsg='4326').geometry.item().centroid.x
     y_mean = chosen_path.to_crs(epsg='4326').geometry.item().centroid.y

     # Create a Folium map centered around the mean of the GPS points
     center = [y_mean,x_mean-.04]
     mymap = folium.Map(location=center, zoom_start=13)

     # Convert GeoDataFrames to GeoJSON
     chosen_path_geojson = chosen_path.to_crs(epsg='4326').to_json()
     shortest_path_geojson = shortest_path.to_crs(epsg='4326').to_json()
     intersection_geojson = intersection.to_crs(epsg='4326').to_json()
     modeled_path_geojson = modeled_path.to_crs(epsg='4326').to_json()

     # Create FeatureGroups for each GeoDataFrame
     chosen_path_fg = FeatureGroup(name='Chosen Path')
     shortest_path_fg = FeatureGroup(name='Shortest Path')
     intersection_fg = FeatureGroup(name='Buffer Intersection',show=False)
     modeled_path_fg = FeatureGroup(name='Modeled Path')

     # Add GeoJSON data to FeatureGroups
     folium.GeoJson(chosen_path_geojson, name='Chosen Path',
                    style_function=lambda x: {'color': '#fc8d62', 'weight': 12}).add_to(chosen_path_fg)
     folium.GeoJson(shortest_path_geojson, name='Shortest Path',
                    style_function=lambda x: {'color': '#66c2a5', 'weight': 8}).add_to(shortest_path_fg)
     folium.GeoJson(intersection_geojson, name='Buffer Intersection',
                    style_function=lambda x: {'color':"gray",'fillColor':"#ffff00","fillOpacity": 0.75}).add_to(intersection_fg)
     folium.GeoJson(modeled_path_geojson, name='Modeled Path',
                    style_function=lambda x: {'color': '#8da0cb','weight': 8}).add_to(modeled_path_fg)

     # Add FeatureGroups to the map
     chosen_path_fg.add_to(mymap)
     shortest_path_fg.add_to(mymap)
     intersection_fg.add_to(mymap)
     modeled_path_fg.add_to(mymap)

     # Add start and end points with play and stop buttons
     start_icon = folium.Icon(color='green',icon='play',prefix='fa')
     end_icon = folium.Icon(color='red',icon='stop',prefix='fa')
     folium.Marker(location=[start_pt.y, start_pt.x],icon=start_icon).add_to(mymap)
     folium.Marker(location=[end_pt.y, end_pt.x],icon=end_icon).add_to(mymap)

     # Add layer control to toggle layers on/off
     folium.LayerControl().add_to(mymap)

     #retrive overlap
     # exact_overlap = gdf.loc[gdf['tripid']==tripid,'shortest_exact_overlap_prop'].item()
     # buffer_overlap = gdf.loc[gdf['tripid']==tripid,'shortest_buffer_overlap'].item()
     row = gdf.loc[gdf['tripid']==tripid].squeeze()

     # Add legend with statistics
     #TODO what happened to duration
     legend_html = f'''
          <div style="position: fixed; 
                    bottom: 5px; left: 5px; width: 300px; height: 400px; 
                    border:2px solid grey; z-index:9999; font-size:14px;
                    background-color: white;
                    opacity: 0.9;">
          &nbsp; <b>Trip ID: {tripid}, User ID: {row['userid']}</b> <br>
          &nbsp; <b> Date: {row['starttime']} </b> <br>
          &nbsp; Start Point &nbsp; <i class="fa fa-play" style="color:green"></i>,
          End Point &nbsp; <i class="fa fa-stop" style="color:red"></i> <br>
          
          &nbsp; Chosen Path &nbsp; <div style="width: 20px; height: 5px; background-color: #fc8d62; display: inline-block;"></div> <br>
          &nbsp; Shortest Path &nbsp; <div style="width: 20px; height: 5px; background-color: #66c2a5; display: inline-block;"></div> <br>
          &nbsp; Modeled Path &nbsp; <div style="width: 20px; height: 5px; background-color: #8da0cb; display: inline-block;"></div> <br>
          &nbsp; Buffer Overlap &nbsp; <div style="width: 20px; height: 10px; background-color: #ffff00; display: inline-block;"></div> <br>

          &nbsp; Percent Detour: {row['percent_detour']:.0f}% <br>
          &nbsp; Shortest Path Overlap: {row['shortest_buffer_overlap']*100:.0f}% <br>
          &nbsp; Modeled Path Overlap: {row['overlap']*100:.0f}% <br>
          &nbsp; Trip Type: {row['trip_type']} <br>
          &nbsp; Length (mi): {row['length_ft']/5280:.0f} <br>
          &nbsp; Age: {row['age']} <br>
          &nbsp; Gender: {row['gender']} <br>
          &nbsp; Income: {row['income']} <br>
          &nbsp; Ethnicity: {row['ethnicity']} <br>
          &nbsp; Cycling Frequency: {row['cyclingfreq']} <br>
          &nbsp; Rider History: {row['rider_history']} <br>
          &nbsp; Rider Type: {row['rider_type']} <br><br>

          </div>
          '''
     mymap.get_root().html.add_child(folium.Element(legend_html))

     # Save the map to an HTML file or display it in a Jupyter notebook
     #mymap.save('map.html')
     # mymap.save('/path/to/save/map.html')  # Use an absolute path if needed
     return mymap  # Uncomment if you are using Jupyter notebook

     #TODO add in the legend with trip info and then we're golden


In [ ]:
test_merge.tripid

In [ ]:
#tripid = 891#30000
tripid = 7257#9806#891
mymap = visualize(test_merge,tripid)
mymap

In [ ]:
gdf.columns

In [ ]:
#random two points appears to work fine
#im thinking it has something to do with the virtual edges?

import random
            source = random.choice(list(pseudo_G.nodes()))
            target = random.choice(list(pseudo_G.nodes()))
            length, node_list = nx.single_source_dijkstra(pseudo_G,source,target,weight='weight')

In [ ]:
        # if ~pseudo_G.has_node(source):
        #     start_coord = trips_df.loc[trips_df['o']==source,['start_lat','start_lon']].iloc[0]
        #     start_coord['geometry'] = Point(start_coord.iloc[0,1],start_coord.iloc[0,0])
        #     start_coord = gpd.GeoDataFrame(start_coord,geometry='geometry',crs='epsg:4326')
        #     start_coord.to_crs(nodes.crs)
        #     source = gpd.sjoin_nearest(start_coord, nodes)['N'].item()